In [1]:
import pandas as pd
import os
import requests

In [2]:
df_pa = pd.read_csv("https://www.fema.gov/api/open/v1/PublicAssistanceFundedProjectsSummaries.csv")
print(df_pa.shape)
df_pa.head()

(170325, 11)


,disasterNumber,declarationDate,incidentType,state,county,applicantName,educationApplicant,numberOfProjects,federalObligatedAmount,hash,lastRefresh
0,1239,1998-08-26T00:00:00.000Z,Severe Storm(s),Texas,Edwards,EDWARDS (COUNTY),0,20,352427.09,d3b0b6e3a5a0acb57f43f8ee7db91beb,2019-04-22T20:01:57.897Z
1,1239,1998-08-26T00:00:00.000Z,Severe Storm(s),Texas,Real,REAL (COUNTY),0,41,283369.22,bbfa953ad93a9c240722246cdea110a9,2019-04-22T20:01:57.901Z
2,1239,1998-08-26T00:00:00.000Z,Severe Storm(s),Texas,Val Verde,VAL VERDE COMMUNITY FOUNDATION,0,1,10080.42,634e53ed887c68de3f4189ef8ee17384,2019-04-22T20:01:57.902Z
3,1239,1998-08-26T00:00:00.000Z,Severe Storm(s),Texas,Statewide,GENERAL SERVICES COMMISSION OF TEXAS,0,1,23875.08,702da47382d16af313c921dbee280dae,2019-04-22T20:01:57.891Z
4,1239,1998-08-26T00:00:00.000Z,Severe Storm(s),Texas,Webb,RIO BRAVO,0,1,11550.00,3837fc90a4798ec67e29d5c689144e0a,2019-04-22T20:01:57.893Z


In [3]:
df_hm = pd.read_csv("https://www.fema.gov/api/open/v1/HazardMitigationAssistanceProjects.csv")
print(df_hm.shape)
df_hm.head()

(27578, 26)


,disasterNumber,region,stateNumberCode,state,countyCode,county,programArea,projectIdentifier,projectType,projectTitle,...,projectAmount,federalShareObligated,programFy,dateInitiallyApproved,dateApproved,dateClosed,tribalIndicator,grantee,hash,lastRefresh
0,1561.0,4,12,Florida,127.0,Volusia,HMGP,DR-1561-0151-R,205.8: Retrofitting Public Structures - Wind,"CITY OF PORT ORANGE, VOLUSIA COUNTY, CYPRESS H...",...,57453.0,43089.62,2004.0,2007-11-13T05:00:00.000Z,2011-02-07T05:00:00.000Z,NaN,NaN,Statewide,bf7230330190c7e1cfe541ad5f259b7f,NaN
1,1332.0,5,55,Wisconsin,41.0,Forest,HMGP,DR-1332-0011-R,401.1: Water and Sanitary Sewer System Protect...,City of Crandon-stormsewer,...,145753.0,109313.00,2000.0,2002-08-21T04:00:00.000Z,2007-03-27T04:00:00.000Z,2007-03-27T04:00:00.000Z,NaN,Statewide,aaffc6ec242829c7fe56eaf6a8bfd242,NaN
2,1401.0,6,40,Oklahoma,79.0,Le Flore,HMGP,DR-1401-0317-P,91.1: Local Multihazard Mitigation Plan,Howe (town) Multi-Hazard Mitigation Plan,...,2000.0,1500.00,2002.0,2005-07-26T04:00:00.000Z,2008-06-04T04:00:00.000Z,2008-03-11T04:00:00.000Z,NaN,Statewide,d60749e0b474b7bd521bfdcf19c82d04,NaN
3,1484.0,5,39,Ohio,63.0,Hancock,HMGP,DR-1484-0008-P,91.1: Local Multihazard Mitigation Plan,Hancock County All Natural Hazard Planning Grant,...,36000.0,27000.00,2003.0,2004-11-16T05:00:00.000Z,2008-02-28T05:00:00.000Z,2008-02-28T05:00:00.000Z,NaN,Statewide,503d425ff0941d2bd8406435974da481,NaN
4,NaN,4,28,Mississippi,141.0,Tishomingo,PDM,PDMC-PL-04-MS-2008-001,91.1: Local Multihazard Mitigation Plan,NE MS Planning and Development District Regio...,...,100000.0,74999.64,2008.0,2008-09-04T04:00:00.000Z,NaN,NaN,NaN,Mississippi Emergency Management Agency,c6f51a121cd806356ee4e0609b97629e,NaN


In [4]:
df_decs = pd.read_csv("https://www.fema.gov/api/open/v1/FemaWebDisasterDeclarations.csv")
print(df_decs.shape)
df_decs.head()

(4145, 14)


,disasterNumber,declarationDate,disasterName,incidentBeginDate,incidentEndDate,declarationType,stateCode,stateName,incidentType,entryDate,updateDate,closeoutDate,hash,lastRefresh
0,3397,2018-02-11T00:00:00.000Z,TROPICAL STORM GITA,2018-02-07T00:00:00.000Z,2018-02-12T00:00:00.000Z,Emergency,AS,American Samoa,Hurricane,2018-02-11T00:00:00.000Z,2018-09-26T00:00:00.000Z,NaN,26ec49d88ca8a9cd2419998ec2a6aabb,2018-09-26T14:20:59.689Z
1,3390,2017-09-18T00:00:00.000Z,HURRICANE MARIA,2017-09-16T00:00:00.000Z,2017-09-22T00:00:00.000Z,Emergency,VI,Virgin Islands of the U.S.,Hurricane,2017-09-18T00:00:00.000Z,2018-09-07T00:00:00.000Z,2018-09-07T00:00:00.000Z,606b8d3fbf7e4bd84b75a2be2fbf4117,2018-09-07T13:42:29.408Z
2,4352,2017-12-20T00:00:00.000Z,SEVERE STORMS AND FLOODING,2017-10-04T00:00:00.000Z,2017-10-06T00:00:00.000Z,Major Disaster,NM,New Mexico,Flood,2017-12-21T00:00:00.000Z,2017-12-21T00:00:00.000Z,NaN,348524eb4c9654eae1543ae0b5da70b8,2018-03-09T18:41:52.621Z
3,3388,2017-09-08T00:00:00.000Z,HURRICANE IRMA,2017-09-04T00:00:00.000Z,2017-10-04T00:00:00.000Z,Emergency,FL,Florida,Hurricane,2017-09-09T00:00:00.000Z,2018-04-03T00:00:00.000Z,2018-03-28T00:00:00.000Z,ac1cc3782349186abe34b8cfe134ef59,2018-04-03T22:01:53.622Z
4,3395,2017-10-08T00:00:00.000Z,HURRICANE NATE,2017-10-07T00:00:00.000Z,2017-10-11T00:00:00.000Z,Emergency,FL,Florida,Hurricane,2017-10-08T00:00:00.000Z,2017-10-13T00:00:00.000Z,NaN,ef18518697366c7663f5767c86ce7e7a,2018-03-09T18:41:52.623Z


In [5]:
petition_to_challenge_states = ['Arizona','Utah','Colorado','Wyoming','Montana','North Dakota','South Dakota','Nebraska','Kansas','Oklahoma','Texas','Louisiana','Arkansas','Missouri','Wisconsin','Indiana','Michigan','Ohio','Mississippi','Alabama','Georgia','Florida','South Carolina','New Jersey','West Virginia','Kentucky']
print(len(petition_to_challenge_states))
oppose_as_amicus_states = ['Nevada']
print(len(oppose_as_amicus_states))
all_oppose_states = petition_to_challenge_states + oppose_as_amicus_states
supporting_as_intervenors_states = ['Washington','Oregon','California','Hawaii','District of Columbia','Maine','New Mexico','Minnesota','Iowa','Illinois','New York','Virginia','Delaware','Vermont','New Hampshire','Rhode Island','Maryland','Massachusetts','Connecticut'] 
print(len(supporting_as_intervenors_states))
not_party_to_litigation_states = ['Alaska','Idaho','Tennessee','North Carolina','Pennsylvania']
print(len(supporting_as_intervenors_states))
all_states_and_DC = all_oppose_states + supporting_as_intervenors_states + not_party_to_litigation_states
print(len(set(all_states_and_DC)))

26
1
19
19
51


In [6]:
# Filtering

In [7]:
# By year

In [8]:
df_decs_decade = df_decs.loc[(df_decs['incidentBeginDate']>="2009-01-01T00:00:00.000Z")&
                         (df_decs['incidentBeginDate']<"2019-01-01T00:00:00.000Z"),'disasterNumber'].copy()

In [9]:
df_decs_decade.min(),df_decs_decade.max()

(1817, 5280)

In [10]:
df_hm_decade = df_hm[df_hm['disasterNumber'].isin(df_decs_decade)]
print(df_hm_decade.shape)

(8037, 26)


In [11]:
df_hm_decade['disasterNumber'].min(),df_hm_decade['disasterNumber'].max()

(1817.0, 5263.0)

In [12]:
df_pa_decade = df_pa[df_pa['disasterNumber'].isin(df_decs_decade)]
print(df_pa_decade.shape)

(69061, 11)


In [13]:
df_pa_decade['disasterNumber'].min(),df_pa_decade['disasterNumber'].max()

(1817, 4418)

In [14]:
# By sub-program of Haz Mit

In [15]:
df_hmgp_decade = df_hm_decade[df_hm_decade['programArea']=='HMGP'].copy()
print(df_hmgp_decade.shape)

(8037, 26)


In [16]:
# By state-clean-power-plan status
df_pa_decade_all_oppose_states = df_pa_decade[df_pa_decade['state'].isin(all_oppose_states)].copy()
print(df_pa_decade_all_oppose_states.shape)

df_hmgp_decade_all_oppose_states = df_hmgp_decade[df_hmgp_decade['state'].isin(all_oppose_states)].copy()
print(df_hmgp_decade_all_oppose_states.shape)

(33665, 11)
(5309, 26)


In [17]:
df_pa_decade_all_oppose_states['state'].nunique(),df_hmgp_decade_all_oppose_states['state'].nunique()

(27, 27)

In [18]:
# Total federal obligated amount across all public assistance projects for declared disasters which began...
# ... between January 1st, 2009 and December 31st, 2018 in states that either petitioned to challenge the clean power plan...
# ... or opposed it as amicus curiae.
df_pa_decade_all_oppose_states['federalObligatedAmount'].sum()

13543341494.25

In [19]:
# Total federal share obligated amount across all hazard mitigation grant program projects for declared disasters which began...
# ... between January 1st, 2009 and December 31st, 2018 in states that either petitioned to challenge the clean power plan...
# ... or opposed it as amicus curiae.
df_hmgp_decade_all_oppose_states['federalShareObligated'].sum()

2007190893.0

Compare to rest of states

In [20]:
print(df_pa_decade['federalObligatedAmount'].sum())
print(df_pa_decade_all_oppose_states['federalObligatedAmount'].sum() / df_pa_decade['federalObligatedAmount'].sum())

43528205946.48001
0.31113943705610514


In [21]:
print(df_hmgp_decade['federalShareObligated'].sum())
print(df_hmgp_decade_all_oppose_states['federalShareObligated'].sum() / df_hmgp_decade['federalShareObligated'].sum())

3989083023.64
0.5031709997272651


In [41]:
df_funding_by_state = pd.DataFrame(data = {'State': all_states_and_DC})
df_funding_by_state.sort_values(by = 'State', inplace = True)
df_funding_by_state.reset_index(inplace = True, drop = True)
df_funding_by_state['Clean Power Plan'] = df_funding_by_state['State'].apply(lambda x: "Oppose" if x in all_oppose_states else "Not Oppose")
df_funding_by_state['Public Assistance'] = df_funding_by_state['State'].apply(lambda x:
                df_pa_decade.loc[df_pa_decade['state']==x,'federalObligatedAmount'].sum())
df_funding_by_state['HMGP'] = df_funding_by_state['State'].apply(lambda x:
                df_hmgp_decade.loc[df_hmgp_decade['state']==x,'federalShareObligated'].sum())
df_funding_by_state['Total'] = df_funding_by_state['Public Assistance'] + df_funding_by_state['HMGP']
df_funding_by_state

,State,Clean Power Plan,Public Assistance,HMGP,Total
0,Alabama,Oppose,3.844514e+08,9.297300e+07,4.774244e+08
1,Alaska,Not Oppose,1.453336e+08,1.664749e+07,1.619811e+08
2,Arizona,Oppose,2.175218e+07,3.863173e+06,2.561535e+07
3,Arkansas,Oppose,4.071761e+08,5.875070e+07,4.659268e+08
4,California,Not Oppose,1.617825e+09,1.250342e+08,1.742859e+09
5,Colorado,Oppose,3.798515e+08,6.526195e+07,4.451134e+08
6,Connecticut,Not Oppose,2.607913e+08,3.731254e+07,2.981039e+08
7,Delaware,Not Oppose,2.044615e+07,1.332941e+06,2.177909e+07
8,District of Columbia,Not Oppose,5.087389e+07,3.139413e+06,5.401330e+07
9,Florida,Oppose,1.843708e+09,1.575132e+08,2.001221e+09


In [24]:
df_funding_by_state.sort_values(by = "Public Assistance", ascending = False).head(10)

,State,Clean Power Plan,Public Assistance,HMGP,Total
32,New York,Not Oppose,1.568205e+10,9.948583e+08,1.667691e+10
30,New Jersey,Oppose,2.404056e+09,3.867785e+08,2.790835e+09
43,Texas,Oppose,2.139857e+09,3.081395e+08,2.447997e+09
9,Florida,Oppose,1.843708e+09,1.575132e+08,2.001221e+09
4,California,Not Oppose,1.617825e+09,1.250342e+08,1.742859e+09
18,Louisiana,Oppose,1.161874e+09,1.158892e+08,1.277763e+09
33,North Carolina,Not Oppose,7.222373e+08,1.015327e+08,8.237699e+08
40,South Carolina,Oppose,6.657998e+08,7.755939e+07,7.433592e+08
36,Oklahoma,Oppose,6.606276e+08,8.353309e+07,7.441607e+08
25,Missouri,Oppose,5.593064e+08,1.848041e+08,7.441105e+08


In [25]:
df_funding_by_state.sort_values(by = "Public Assistance", ascending = False).head(10).to_csv("top_public_assistance_funding", index = False)

In [26]:
df_funding_by_state.sort_values(by = "HMGP", ascending = False).head(10)

,State,Clean Power Plan,Public Assistance,HMGP,Total
32,New York,Not Oppose,1.568205e+10,9.948583e+08,1.667691e+10
30,New Jersey,Oppose,2.404056e+09,3.867785e+08,2.790835e+09
43,Texas,Oppose,2.139857e+09,3.081395e+08,2.447997e+09
25,Missouri,Oppose,5.593064e+08,1.848041e+08,7.441105e+08
9,Florida,Oppose,1.843708e+09,1.575132e+08,2.001221e+09
4,California,Not Oppose,1.617825e+09,1.250342e+08,1.742859e+09
18,Louisiana,Oppose,1.161874e+09,1.158892e+08,1.277763e+09
33,North Carolina,Not Oppose,7.222373e+08,1.015327e+08,8.237699e+08
34,North Dakota,Oppose,4.398036e+08,9.412809e+07,5.339317e+08
0,Alabama,Oppose,3.844514e+08,9.297300e+07,4.774244e+08


In [27]:
df_funding_by_state.sort_values(by = "HMGP", ascending = False).head(10).to_csv("top_hmgp_funding", index = False)

In [28]:
df_funding_by_state.sort_values(by = "Total", ascending = False).head(10)

,State,Clean Power Plan,Public Assistance,HMGP,Total
32,New York,Not Oppose,1.568205e+10,9.948583e+08,1.667691e+10
30,New Jersey,Oppose,2.404056e+09,3.867785e+08,2.790835e+09
43,Texas,Oppose,2.139857e+09,3.081395e+08,2.447997e+09
9,Florida,Oppose,1.843708e+09,1.575132e+08,2.001221e+09
4,California,Not Oppose,1.617825e+09,1.250342e+08,1.742859e+09
18,Louisiana,Oppose,1.161874e+09,1.158892e+08,1.277763e+09
33,North Carolina,Not Oppose,7.222373e+08,1.015327e+08,8.237699e+08
36,Oklahoma,Oppose,6.606276e+08,8.353309e+07,7.441607e+08
25,Missouri,Oppose,5.593064e+08,1.848041e+08,7.441105e+08
40,South Carolina,Oppose,6.657998e+08,7.755939e+07,7.433592e+08


In [29]:
df_funding_by_state.sort_values(by = "Total", ascending = False).head(10).to_csv("top_total_funding", index = False)

Add GHG and GHG per capita (for now, just CO2)

In [30]:
# https://www.eia.gov/environment/emissions/state/
url = 'https://www.eia.gov/environment/emissions/state/excel/summary_2016.xlsx'
df_co2_emissions = pd.read_excel(url, skiprows = 2, index_col = 0)
print(df_co2_emissions.shape)
df_co2_emissions.head()

(57, 29)


,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,...,2009,2010,2011,2012,2013,2014,2015,2016,Percent,Absolute
Million metric tons of carbon dioxide,,,,,,,,,,,,,,,,,,,,,
Alabama,111.092122,115.466992,122.726107,127.038855,125.120631,132.958723,139.035875,135.526720,135.547782,138.156619,...,121.948916,134.614539,131.646689,125.112394,122.512730,124.751678,121.682353,115.721763,0.041674,4.629641
Alaska,35.054100,35.685200,37.298110,36.969789,36.825473,41.503806,42.450252,42.393320,43.754118,44.377008,...,38.588620,39.465576,39.306465,38.829165,36.659247,35.795496,36.696731,35.101593,0.001355,0.047493
Arizona,63.491486,64.398631,67.532477,69.649832,72.475134,67.372781,69.177065,72.138275,77.575691,81.560618,...,94.836579,96.637581,94.617175,92.780880,96.450602,94.371266,92.111361,87.484845,0.377899,23.993359
Arkansas,51.477171,50.380231,52.232946,51.171105,55.153835,58.564112,61.175920,59.981530,61.743580,64.051579,...,62.419653,66.723731,68.145755,67.036206,69.343168,69.892555,60.011481,62.754316,0.219071,11.277145
California,362.964725,350.440893,356.370525,344.135973,360.699934,349.382009,347.889790,349.521529,362.299332,366.313675,...,373.800883,367.817653,353.224905,358.604029,360.716723,357.499077,365.409907,363.342966,0.001042,0.378240


In [31]:
key = open('census_api_key.txt','r').read()

In [32]:
population = "B01001_001E"
base_url = "https://api.census.gov/data/2016/acs/acs5?"
end_url = "get=NAME,%s&for=state:*&key=%s" % (population, key)

response = requests.get(base_url+end_url)
print(response)
response_json = response.json()
df_population = pd.DataFrame(response_json[1:],
                                columns = response_json[0])
df_population.set_index("NAME", inplace = True)
df_population.head()

<Response [200]>


,B01001_001E,state
NAME,,
Alabama,4841164,01
Alaska,736855,02
Arizona,6728577,04
Arkansas,2968472,05
California,38654206,06


In [42]:
# df_funding_by_state['CO2 2007-16'] = df_funding_by_state['State'].apply(lambda x:
#             df_co2_emissions.loc[x,2007:2017].sum())
df_funding_by_state['CO2 2016 (Millions of Metric Tons)'] = df_funding_by_state['State'].apply(lambda x:
            df_co2_emissions.loc[x,2016])
df_funding_by_state['CO2 2016 (Metric Tons)'] = df_funding_by_state['CO2 2016 (Millions of Metric Tons)']*(10**6)
df_funding_by_state['Pop 2016'] = df_funding_by_state['State'].apply(lambda x:
        df_population.loc[x,population]).apply(int)
df_funding_by_state['CO2 Emissions (Metric Tons Per Capita) 2016'] = df_funding_by_state['CO2 2016 (Metric Tons)'] / df_funding_by_state['Pop 2016']
df_funding_by_state.head()

,State,Clean Power Plan,Public Assistance,HMGP,Total,CO2 2016 (Millions of Metric Tons),CO2 2016 (Metric Tons),Pop 2016,CO2 Emissions (Metric Tons Per Capita) 2016
0,Alabama,Oppose,3.844514e+08,9.297300e+07,4.774244e+08,115.721763,1.157218e+08,4841164,23.903706
1,Alaska,Not Oppose,1.453336e+08,1.664749e+07,1.619811e+08,35.101593,3.510159e+07,736855,47.637043
2,Arizona,Oppose,2.175218e+07,3.863173e+06,2.561535e+07,87.484845,8.748484e+07,6728577,13.001983
3,Arkansas,Oppose,4.071761e+08,5.875070e+07,4.659268e+08,62.754316,6.275432e+07,2968472,21.140276
4,California,Not Oppose,1.617825e+09,1.250342e+08,1.742859e+09,363.342966,3.633430e+08,38654206,9.399830


In [43]:
#df_funding_by_state.to_csv("fema_funding_by_state", index = False)